In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from patchify import patchify, unpatchify
from tensorflow.keras.models import load_model
from sklearn.metrics import jaccard_score, precision_score, recall_score
from sklearn.metrics import f1_score as f1_metric
import pandas as pd
from datetime import datetime

from preprocess import crop, padder, crop_to_coordinates
from helpers import f1

def inference_single_model(model, image_dir, time_steps=15, kernel_size=5, patch_size=256):
    """
    Run inference for a single model and return metrics.
    """
    # Load image file names
    files = sorted([f for f in os.listdir(image_dir) if f.endswith('.png')])
    mask_dir = image_dir

    # Metrics storage
    all_f1, all_iou, all_precision, all_recall = [], [], [], []

    num_sequences = len(files) // time_steps
    for seq_idx in range(num_sequences):
        seq_files = files[seq_idx * time_steps:(seq_idx + 1) * time_steps]
        frames_padded = []
        stats, centroids = None, None

        # Preprocess each frame in the sequence
        for file in seq_files:
            img_path = os.path.join(image_dir, file)
            image = cv2.imread(img_path, 0)  # Grayscale
            image_cropped, stats, centroids = crop(image, kernel_size=kernel_size)
            image_color = cv2.cvtColor(image_cropped, cv2.COLOR_GRAY2RGB)
            image_np = image_color / 255.0
            image_padded = padder(image_np, patch_size=patch_size)
            frames_padded.append(image_padded)

        frames_padded = np.array(frames_padded)  # (time_steps, H, W, 3)
        H, W, _ = frames_padded[0].shape

        # Patchify all frames
        patches_seq = []
        for t in range(time_steps):
            patches = patchify(frames_padded[t], (patch_size, patch_size, 1), step=patch_size)
            patches_seq.append(patches)

        patches_seq = np.stack(patches_seq, axis=2)
        num_x, num_y = patches_seq.shape[:2]

        # Reshape to (num_patches, time_steps, patch_size, patch_size, 3)
        patches_seq = patches_seq.reshape(-1, time_steps, patch_size, patch_size, 1)

        # Predict masks for all time steps
        predicted_patches = model.predict(patches_seq, verbose=0)

        # Reshape back to (num_x, num_y, time_steps, patch_size, patch_size)
        predicted_patches = predicted_patches.reshape(num_x, num_y, time_steps, patch_size, patch_size)

        # Process each frame
        for t in range(time_steps):
            predicted_mask = unpatchify(predicted_patches[:, :, t, :, :], (H, W))
            prediction_bool = (predicted_mask > 0.5).astype(np.uint8)

            # Load corresponding ground truth mask
            mask_filename = seq_files[t].replace('.png', '.tif')
            mask_path = os.path.join(mask_dir, mask_filename)
            if not os.path.exists(mask_path):
                print(f"  Mask for {seq_files[t]} not found, skipping frame...")
                continue

            mask = cv2.imread(mask_path, 0)
            mask = crop_to_coordinates(mask, stats, centroids)
            mask = padder(mask, patch_size=patch_size)
            mask_bool = (mask > 0.5).astype(np.uint8)

            if np.sum(mask_bool) == 0:
                print(f"  Skipping frame {seq_files[t]} as ground truth has no positive pixels.")
                continue

            # Compute metrics
            pred_flat = prediction_bool.flatten()
            mask_flat = mask_bool.flatten()
            f1_val = f1_metric(mask_flat, pred_flat)
            iou_val = jaccard_score(mask_flat, pred_flat)
            precision_val = precision_score(mask_flat, pred_flat, zero_division=0)
            recall_val = recall_score(mask_flat, pred_flat, zero_division=0)

            all_f1.append(f1_val)
            all_iou.append(iou_val)
            all_precision.append(precision_val)
            all_recall.append(recall_val)

    # Calculate average metrics
    metrics = {
        'f1': np.mean(all_f1) if all_f1 else 0.0,
        'iou': np.mean(all_iou) if all_iou else 0.0,
        'precision': np.mean(all_precision) if all_precision else 0.0,
        'recall': np.mean(all_recall) if all_recall else 0.0,
        'num_frames': len(all_f1)
    }
    
    return metrics

def pipeline_all_models(models_dir, image_dir, time_steps=15, kernel_size=5, patch_size=256, 
                       model_extensions=['.h5', '.keras'], save_results=True):
    """
    Run inference on all models in a directory and compile results.
    
    Args:
        models_dir: Directory containing all model files
        image_dir: Directory containing test images
        time_steps: Number of time steps for LSTM models
        kernel_size: Kernel size for preprocessing
        patch_size: Patch size for patchifying
        model_extensions: List of model file extensions to look for
        save_results: Whether to save results to CSV
    
    Returns:
        DataFrame with results for all models
    """
    
    # Find all model files
    model_files = []
    for ext in model_extensions:
        model_files.extend([f for f in os.listdir(models_dir) if f.endswith(ext)])
    
    if not model_files:
        print(f"No model files found in {models_dir} with extensions {model_extensions}")
        return None
    
    print(f"Found {len(model_files)} models to evaluate:")
    for mf in model_files:
        print(f"  - {mf}")
    print("\n" + "="*60 + "\n")
    
    # Store results
    results = []
    
    # Process each model
    for idx, model_file in enumerate(model_files, 1):
        model_path = os.path.join(models_dir, model_file)
        model_name = os.path.splitext(model_file)[0]
        
        try:
            # Load model
            model = load_model(model_path, custom_objects={"f1": f1})
            
            # Run inference
            metrics = inference_single_model(
                model, 
                image_dir, 
                time_steps=time_steps,
                kernel_size=kernel_size,
                patch_size=patch_size
            )
            
            # Add model info to metrics
            metrics['model_name'] = model_name
            metrics['model_file'] = model_file
            
            # Print results for this model
            print(f"\n  Results for {model_name}:")
            print(f"    F1 Score:  {metrics['f1']:.4f}")
            print(f"    mIoU:      {metrics['iou']:.4f}")
            print(f"    Precision: {metrics['precision']:.4f}")
            print(f"    Recall:    {metrics['recall']:.4f}")
            print(f"    Frames:    {metrics['num_frames']}")
            
            results.append(metrics)
            
            # Clear model from memory
            del model
            
        except Exception as e:
            print(f"  ERROR processing {model_name}: {str(e)}")
            results.append({
                'model_name': model_name,
                'model_file': model_file,
                'f1': np.nan,
                'iou': np.nan,
                'precision': np.nan,
                'recall': np.nan,
                'num_frames': 0,
                'error': str(e)
            })
        
        print("\n" + "="*60 + "\n")
    
    # Create DataFrame with results
    df_results = pd.DataFrame(results)
    
    # Reorder columns for better readability
    column_order = ['model_name', 'f1', 'iou', 'precision', 'recall', 'num_frames', 'model_file']
    if 'error' in df_results.columns:
        column_order.append('error')
    df_results = df_results[column_order]
    
    # Sort by F1 score (descending)
    df_results = df_results.sort_values('f1', ascending=False)
    
    # Save results if requested
    if save_results:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"lstm_resnet_results_{timestamp}.csv"
        df_results.to_csv(output_file, index=False)

    
    # Print summary
    print("\n" + "="*60)
    print("SUMMARY OF ALL MODELS")
    print("="*60)
    print(df_results.to_string(index=False))
    
    # Print best performing model
    if not df_results.empty and not df_results['f1'].isna().all():
        best_model = df_results.iloc[0]
        print("\n" + "="*60)
        print(f"  F1 Score:  {best_model['f1']:.4f}")
        print(f"  mIoU:      {best_model['iou']:.4f}")
        print("="*60)
    
    return df_results

# Configuration
MODELS_DIR = "models"  # Directory containing all your trained models
IMAGE_DIR = "../LSTM-Unet/data_v3_processed/test"
TIME_STEPS = 15
KERNEL_SIZE = 5
PATCH_SIZE = 256

# Run inference on all models
results_df = pipeline_all_models(
    models_dir=MODELS_DIR,
    image_dir=IMAGE_DIR,
    time_steps=TIME_STEPS,
    kernel_size=KERNEL_SIZE,
    patch_size=PATCH_SIZE,
    model_extensions=['.h5'],
    save_results=True
)

Found 5 models to evaluate:
  - lstm_vit_run3.h5
  - lstm_vit_run1.h5
  - lstm_vit_run4.h5
  - lstm_vit_run5.h5
  - lstm_vit_run2.h5




2025-08-21 14:37:58.922170: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906


  Skipping frame 28_05_01.png as ground truth has no positive pixels.

  Results for lstm_vit_run3:
    F1 Score:  0.7648
    mIoU:      0.6193
    Precision: 0.9065
    Recall:    0.6621
    Frames:    14


  Skipping frame 28_05_01.png as ground truth has no positive pixels.

  Results for lstm_vit_run1:
    F1 Score:  0.7775
    mIoU:      0.6361
    Precision: 0.9250
    Recall:    0.6715
    Frames:    14


  Skipping frame 28_05_01.png as ground truth has no positive pixels.

  Results for lstm_vit_run4:
    F1 Score:  0.7766
    mIoU:      0.6349
    Precision: 0.9181
    Recall:    0.6738
    Frames:    14


  Skipping frame 28_05_01.png as ground truth has no positive pixels.

  Results for lstm_vit_run5:
    F1 Score:  0.7719
    mIoU:      0.6288
    Precision: 0.9217
    Recall:    0.6644
    Frames:    14




  Skipping frame 28_05_01.png as ground truth has no positive pixels.

  Results for lstm_vit_run2:
    F1 Score:  0.0000
    mIoU:      0.0000
    Precision: 0.0000
    Recall:    0.0000
    Frames:    14



SUMMARY OF ALL MODELS
   model_name       f1      iou  precision   recall  num_frames       model_file
lstm_vit_run1 0.777511 0.636110   0.924989 0.671534          14 lstm_vit_run1.h5
lstm_vit_run4 0.776620 0.634950   0.918117 0.673817          14 lstm_vit_run4.h5
lstm_vit_run5 0.771882 0.628756   0.921687 0.664412          14 lstm_vit_run5.h5
lstm_vit_run3 0.764827 0.619317   0.906478 0.662080          14 lstm_vit_run3.h5
lstm_vit_run2 0.000000 0.000000   0.000000 0.000000          14 lstm_vit_run2.h5

  F1 Score:  0.7775
  mIoU:      0.6361
